<a href="https://colab.research.google.com/github/Toshman-hub/PulmoNet/blob/main/PulmoNet_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Loading and preprocessing our dataset
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
# loading training data
X_train = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical_crossentropy')

# loading testing data
test_datagen = ImageDataGenerator(rescale=1./255)
y_train = train_datagen.flow_from_directory(
       'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical_crossentropy')


# hyperparameters Defination
input_shape = (64, 64, 1)
num_classes = 4
learning_rate = 0.001
batch_size = 32
epochs = 10

# Defining the WRN architecture
def wide_residual_network(input_shape, num_classes, depth, width, dropout_rate):
    model = models.Sequential()

    model.add(layers.Conv2D(width, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.Activation('relu'))

    for _ in range(depth - 1):
        model.add(layers.Conv2D(width, (3, 3), padding='same'))
        model.add(layers.Activation('relu'))
        model.add(layers.Dropout(dropout_rate))

    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Definining our model
model = wide_residual_network(input_shape, num_classes, depth=4, width=64, dropout_rate=0.3)
optimizer = optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Performing n-fold cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train.argmax(axis=1))):
    print(f"Fold {fold + 1}")

    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Converting labels to one-hot encoding
    y_train_fold_onehot = tf.keras.utils.to_categorical(y_train_fold, num_classes)
    y_val_fold_onehot = tf.keras.utils.to_categorical(y_val_fold, num_classes)

    # Training the model
    model.fit(X_train_fold, y_train_fold_onehot, batch_size=batch_size, epochs=epochs, validation_data=(X_val_fold, y_val_fold_onehot))

    # Evaluating our model
    y_val_pred = model.predict(X_val_fold)
    y_val_pred_classes = np.argmax(y_val_pred, axis=1)
    y_val_true_classes = np.argmax(y_val_fold_onehot, axis=1)

    accuracy = accuracy_score(y_val_true_classes, y_val_pred_classes)
    precision = precision_score(y_val_true_classes, y_val_pred_classes, average='weighted')
    recall = recall_score(y_val_true_classes, y_val_pred_classes, average='weighted')
    f1 = f1_score(y_val_true_classes, y_val_pred_classes, average='weighted')

    print(f"Fold {fold + 1} - Validation Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")
    cm = confusion_matrix(y_val_true_classes, y_val_pred_classes)
    print(f"Confusion Matrix:\n{cm}\n")
